# Implementation of Chatbot using NLP


In [2]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Importing neccesary Libraries
import nltk
import random
import os
import ssl
import streamlit as s
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt') 

[nltk_data] Downloading package punkt to C:\Users\DHRUVIN
[nltk_data]     PATEL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "What's up?", "How are you?", "Hey there"],
        "responses": ["Hey there!", "Hello! How can I assist you?", "Hi! What's on your mind?", "Hey! Hope you're having a great day!"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care", "Catch you later"],
        "responses": ["Goodbye! Have a great day!", "See you later!", "Take care!", "Until next time!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "I appreciate it", "Much appreciated"],
        "responses": ["You're very welcome!", "No problem at all!", "Happy to help!", "Glad I could assist you!"]
    },
    {
        "tag": "about",
        "patterns": ["Who are you?", "What can you do?", "Tell me about yourself", "What's your purpose?"],
        "responses": ["I'm a chatbot designed to assist you!", "I'm here to answer your questions and help you with various topics.", "Think of me as your virtual assistant, always ready to help!"]
    },
    {
        "tag": "help",
        "patterns": ["I need help", "Can you help me?", "What should I do?", "Help me"],
        "responses": ["Of course! What do you need help with?", "I'm here to assist. Let me know what you need!", "How can I make things easier for you today?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you?", "What's your age?", "When were you created?"],
        "responses": ["I don’t age like humans, but I’m as fresh as the latest update!", "I exist in the digital world, so time is just a concept to me!", "Let's just say I was born in the cloud!"]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like?", "How's the weather today?", "Is it raining outside?"],
        "responses": ["I can't check real-time weather, but you can try a weather app!", "For accurate weather updates, I recommend checking a weather website!", "I wish I could tell you, but checking a weather app might be better!"]
    },
    {
        "tag": "joke",
        "patterns": ["Tell me a joke", "Make me laugh", "Say something funny"],
        "responses": ["Why don’t skeletons fight each other? They don’t have the guts!", "I told my laptop I needed a break, and now it won’t stop sending me vacation ads!", "Parallel lines have so much in common. It’s a shame they’ll never meet."]
    },
    {
        "tag": "motivation",
        "patterns": ["Motivate me", "I need motivation", "Give me some inspiration"],
        "responses": ["The best time to start was yesterday. The next best time is now!", "You are capable of amazing things. Keep going!", "Success starts with the decision to try."]
    },
    {
        "tag": "time",
        "patterns": ["What time is it?", "Can you tell me the time?", "What's the current time?"],
        "responses": ["I can’t check real-time clocks, but your device should have the answer!", "Time is precious, don’t waste it! Check your clock and make the most of your day!", "Your watch might be better at this, but I’m sure it’s a great time to do something productive!"]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget?", "What's a good budgeting strategy?", "How do I create a budget?"],
        "responses": ["A simple way to budget is using the 50/30/20 rule: 50% for needs, 30% for wants, and 20% for savings.", "Start by tracking your expenses and income, then allocate money accordingly.", "Budgeting is about balance—save, spend wisely, and invest in your future!"]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score?", "How do I check my credit score?", "How can I improve my credit score?"],
        "responses": ["A credit score is a number that reflects your financial reliability. The higher, the better!", "You can check your credit score using services like Credit Karma or your bank’s website.", "To improve your credit score, pay bills on time, reduce debt, and avoid unnecessary credit inquiries."]
    },
    {
        "tag": "programming",
        "patterns": ["Can you teach me programming?", "How do I learn to code?", "What are the best programming languages?"],
        "responses": ["Python is a great start for beginners!", "Practice makes perfect—start with small projects and build your way up!", "JavaScript, Python, and Java are among the most popular languages."]
    },
    {
        "tag": "travel",
        "patterns": ["Where should I travel?", "Suggest me a good travel destination", "Best places to visit"],
        "responses": ["It depends on your preferences! Mountains, beaches, or city adventures?", "Paris, Tokyo, and Bali are top travel destinations!", "Looking for adventure? Try skydiving in Dubai or hiking in the Himalayas!"]
    },
    {
        "tag": "fitness",
        "patterns": ["How to stay fit?", "Give me fitness tips", "Best workouts for weight loss"],
        "responses": ["Exercise daily and maintain a balanced diet!", "Consistency is key—find a routine that works for you!", "Try a mix of strength training and cardio for best results!"]
    },
    {
        "tag": "technology",
        "patterns": ["Tell me about AI", "What is machine learning?", "How does deep learning work?"],
        "responses": ["AI is the future! It helps machines learn and make decisions.", "Machine learning is a subset of AI where computers learn from data.", "Deep learning uses neural networks to mimic human thinking!"]
    },
    {
        "tag": "education",
        "patterns": ["How can I improve my learning?", "Give me study tips", "Best way to study?"],
        "responses": ["Stay consistent, take notes, and practice regularly!", "Use active recall and spaced repetition for better retention!", "Break your study time into focused sessions with short breaks!"]
    },
    {
        "tag": "mental_health",
        "patterns": ["I'm feeling sad", "I need motivation", "How to deal with stress?"],
        "responses": ["It's okay to feel this way. Take deep breaths and talk to someone!", "You're stronger than you think! Keep going!", "Try meditation, exercise, or talking to a friend—it helps a lot!"]
    },
    {
        "tag": "entertainment",
        "patterns": ["Suggest me a movie", "Best TV series?", "What should I watch?"],
        "responses": ["How about watching Inception if you love mind-bending movies?", "Breaking Bad is a must-watch if you enjoy crime dramas!", "Try a comedy show like Brooklyn Nine-Nine for a good laugh!"]
    },
    {
        "tag": "gaming",
        "patterns": ["What are the best video games?", "Suggest me a game", "Which game should I play?"],
        "responses": ["If you love action, try Call of Duty or GTA!", "For strategy lovers, Age of Empires is a great pick!", "Minecraft is perfect if you love creativity and adventure!"]
    },
    {
        "tag": "fun_facts",
        "patterns": ["Tell me a fun fact", "Give me an interesting fact", "Random fact?"],
        "responses": ["Did you know honey never spoils? Archaeologists found 3000-year-old honey that was still good!", "A single cloud can weigh more than a million pounds!", "Bananas are berries, but strawberries aren’t!"]
    },
    {
        "tag": "sports",
        "patterns": ["Who is the best football player?", "Tell me about cricket", "Latest sports news?"],
        "responses": ["Messi and Ronaldo are considered two of the greatest footballers ever!", "Cricket is a popular sport played with a bat and ball, loved in countries like India and Australia!", "You can check ESPN or Sports Illustrated for the latest sports news!"]
    }
]


In [10]:
#Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocessing the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)


In [11]:
#training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [12]:
#Python function to chat with the chatbot
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [13]:
# Cheking our Chatbot
user_input = "What is your age ?"
response = chatbot(user_input)
print(response)

Let's just say I was born in the cloud!


In [14]:
# Cheking our Chatbot
user_input = "Hello"
response = chatbot(user_input)
print(response)

Hey! Hope you're having a great day!


In [24]:
# Cheking our Chatbot
user_input = "Which movie should I watch?"
response = chatbot(user_input)
print(response)

Try a comedy show like Brooklyn Nine-Nine for a good laugh!


In [25]:
# Cheking our Chatbot
user_input = "Goodbye"
response = chatbot(user_input)
print(response)

Until next time!


In [26]:
# Cheking our Chatbot
user_input = "budget"
response = chatbot(user_input)
print(response)

Start by tracking your expenses and income, then allocate money accordingly.


In [29]:
# Cheking our Chatbot
user_input = "fact"
response = chatbot(user_input)
print(response)

A single cloud can weigh more than a million pounds!


In [30]:
# Cheking our Chatbot
user_input = "football"
response = chatbot(user_input)
print(response)

Hi! What's on your mind?
